# ============================================================
# 🏠 COMPILATION DES BASES LOGEMENT (2015–2022)
# ============================================================


Dans cette feuile on va : 
- Importer toutes les données
- Compiler les données dans un seul fichier pour une pluriannualité 
- Formater les noms des variables et calculer d'autres variables si nécéssaire


In [1]:
import pandas as pd
import numpy as np
import glob
import os


In [2]:
# -----------------------------------------------
# Paramètres
# -----------------------------------------------

annees = [2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013]
zone = "LIBGEO"  # zone d'agrégation
zone_f = []       # filtres éventuels
dep_filtre = ["30", "34"] # exemple : Gard

In [3]:
# -----------------------------------------------
# Chargement des fichiers Excel logement
# -----------------------------------------------
def charger_fichier_logement(fichier, sheet, annee, skip=5):
    df = pd.read_excel(fichier, sheet_name=sheet, skiprows=skip)
    df["AN"] = annee
    return df

In [4]:
# Dictionnaire des fichiers et années (ajuster selon ton dossier)
fichiers = {
    2022: ("../DATA/base-cc-logement-2022.xlsx", "COM_2022"),
    2021: ("../DATA/base-cc-logement-2021.xlsx", "COM_2021"),
    2020: ("../DATA/base-cc-logement-2020.xlsx", "COM_2020"),
    2019: ("../DATA/base-cc-logement-2019.xlsx", "COM_2019"),
    2018: ("../DATA/base-cc-logement-2018.xlsx", "COM_2018"),
    2017: ("../DATA/base-cc-logement-2017.xlsx", "COM_2017"),
    2016: ("../DATA/base-cc-logement-2016.xls", "COM_2016"),
    2015: ("../DATA/base-cc-logement-2015.xls", "COM_2015"),
    2014: ("../DATA/base-cc-logement-2020.xlsx", "COM_2014"),
    2013: ("../DATA/base-cc-logement-2019.xlsx", "COM_2013"),
}



In [5]:
import os
print("Répertoire courant :", os.getcwd())


Répertoire courant : /Users/aboubacarsiddikhamadoukaka/Documents/M2/S1/open_data/OpenData/notebooks


In [6]:
bases = []
for annee, (fichier, sheet) in fichiers.items():
    df = charger_fichier_logement(fichier, sheet, annee)
    # nettoyage des préfixes Pxx_
    df.columns = df.columns.str.replace(f"P{str(annee)[-2:]}_", "", regex=False)
    bases.append(df)
    

In [29]:
# -----------------------------------------------
# Fusion de toutes les bases
# -----------------------------------------------
histo_log = pd.concat(bases, ignore_index=True)

In [30]:
# Calcul de la variable RP_LOCPRIV qui est le nombre de résidences principales en logements locatifs privés
histo_log= histo_log.assign(RP_LOCPRIV=lambda x: x['RP'] - x['RP_LOCHLMV'])

In [31]:
# Arrondir les variables numériques à l'entier le plus proche
variables_numeriques = histo_log.select_dtypes(include=[np.number]).columns
histo_log[variables_numeriques] = histo_log[variables_numeriques].round(0).astype('Int64')


In [33]:
histo_log = histo_log.assign(
    Plog_RP = round(100 * histo_log["RP"] / histo_log["LOG"], 1),
    Plog_RS = round(100 * histo_log["RSECOCC"] / histo_log["LOG"], 1),
    Plog_VAC = round(100 * histo_log["LOGVAC"] / histo_log["LOG"], 1),
    Plog_RP_LOCHLM = round(100 * histo_log["RP_LOCHLMV"] / histo_log["LOG"], 1),
    Plog_RP_LOCPRIV = round(100 * histo_log["RP_LOCPRIV"] / histo_log["LOG"], 1),
    Plog_RP_LOC = round(100 * histo_log["RP_LOC"] / histo_log["LOG"], 1),
    Plog_RP_PROP = round(100 * histo_log["RP_PROP"] / histo_log["LOG"], 1),
    Plog_RP_GRAT = round(100 * histo_log["RP_GRAT"] / histo_log["LOG"], 1),

    Prp_RP_LOCHLM = round(100 * histo_log["RP_LOCHLMV"] / histo_log["RP"], 1),
    Prp_RP_LOCPRIV = round(100 * histo_log["RP_LOCPRIV"] / histo_log["RP"], 1),
    Prp_RP_LOC = round(100 * histo_log["RP_LOC"] / histo_log["RP"], 1),
    Prp_RP_PROP = round(100 * histo_log["RP_PROP"] / histo_log["RP"], 1),
    Prp_RP_GRAT = round(100 * histo_log["RP_GRAT"] / histo_log["RP"], 1)
)


In [34]:
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("C2")]
histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("C1")]


In [25]:
#histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("RPMAISON_AC")]
#histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("RPAPPART_AC")]

In [26]:
#histo_log = histo_log.loc[:, ~histo_log.columns.str.startswith("MEN_")]

In [35]:
print("Colonnes disponibles :", histo_log.columns.tolist())

Colonnes disponibles : ['CODGEO', 'REG', 'DEP', 'LIBGEO', 'LOG', 'RP', 'RSECOCC', 'LOGVAC', 'MAISON', 'APPART', 'RP_1P', 'RP_2P', 'RP_3P', 'RP_4P', 'RP_5PP', 'NBPI_RP', 'RPMAISON', 'NBPI_RPMAISON', 'RPAPPART', 'NBPI_RPAPPART', 'RP_ACHTOT', 'RP_ACH1919', 'RP_ACH1945', 'RP_ACH1970', 'RP_ACH1990', 'RP_ACH2005', 'RP_ACH2019', 'RPMAISON_ACH1919', 'RPMAISON_ACH1945', 'RPMAISON_ACH1970', 'RPMAISON_ACH1990', 'RPMAISON_ACH2005', 'RPMAISON_ACH2019', 'RPAPPART_ACH1919', 'RPAPPART_ACH1945', 'RPAPPART_ACH1970', 'RPAPPART_ACH1990', 'RPAPPART_ACH2005', 'RPAPPART_ACH2019', 'MEN', 'MEN_ANEM0002', 'MEN_ANEM0204', 'MEN_ANEM0509', 'MEN_ANEM10P', 'MEN_ANEM1019', 'MEN_ANEM2029', 'MEN_ANEM30P', 'PMEN', 'PMEN_ANEM0002', 'PMEN_ANEM0204', 'PMEN_ANEM0509', 'PMEN_ANEM10P', 'NBPI_RP_ANEM0002', 'NBPI_RP_ANEM0204', 'NBPI_RP_ANEM0509', 'NBPI_RP_ANEM10P', 'RP_PROP', 'RP_LOC', 'RP_LOCHLMV', 'RP_GRAT', 'NPER_RP', 'NPER_RP_PROP', 'NPER_RP_LOC', 'NPER_RP_LOCHLMV', 'NPER_RP_GRAT', 'ANEM_RP', 'ANEM_RP_PROP', 'ANEM_RP_LOC', 

In [36]:
histo_log.head()


,CODGEO,REG,DEP,LIBGEO,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,...,Plog_RP_LOCHLM,Plog_RP_LOCPRIV,Plog_RP_LOC,Plog_RP_PROP,Plog_RP_GRAT,Prp_RP_LOCHLM,Prp_RP_LOCPRIV,Prp_RP_LOC,Prp_RP_PROP,Prp_RP_GRAT
0,01001,84,01,L'Abergement-Clémenciat,379,354,11,14,369,9,...,3.7,89.7,12.9,79.9,0.5,4.0,96.0,13.8,85.6,0.6
1,01002,84,01,L'Abergement-de-Varey,175,121,41,13,173,2,...,0.0,69.1,6.9,60.6,1.7,0.0,100.0,9.9,87.6,2.5
2,01004,84,01,Ambérieu-en-Bugey,7973,7107,125,741,3579,4312,...,18.7,70.4,47.3,40.2,1.6,21.0,79.0,53.1,45.1,1.8
3,01005,84,01,Ambérieux-en-Dombes,921,815,8,99,674,244,...,8.4,80.1,22.4,64.5,1.6,9.4,90.6,25.3,72.9,1.8
4,01006,84,01,Ambléon,71,57,8,6,68,3,...,1.4,78.9,21.1,57.7,1.4,1.8,98.2,26.3,71.9,1.8


In [37]:
print("Colonnes disponibles :", histo_log.columns.tolist())
histo_log.head()
histo_log.info()
histo_log.shape

Colonnes disponibles : ['CODGEO', 'REG', 'DEP', 'LIBGEO', 'LOG', 'RP', 'RSECOCC', 'LOGVAC', 'MAISON', 'APPART', 'RP_1P', 'RP_2P', 'RP_3P', 'RP_4P', 'RP_5PP', 'NBPI_RP', 'RPMAISON', 'NBPI_RPMAISON', 'RPAPPART', 'NBPI_RPAPPART', 'RP_ACHTOT', 'RP_ACH1919', 'RP_ACH1945', 'RP_ACH1970', 'RP_ACH1990', 'RP_ACH2005', 'RP_ACH2019', 'RPMAISON_ACH1919', 'RPMAISON_ACH1945', 'RPMAISON_ACH1970', 'RPMAISON_ACH1990', 'RPMAISON_ACH2005', 'RPMAISON_ACH2019', 'RPAPPART_ACH1919', 'RPAPPART_ACH1945', 'RPAPPART_ACH1970', 'RPAPPART_ACH1990', 'RPAPPART_ACH2005', 'RPAPPART_ACH2019', 'MEN', 'MEN_ANEM0002', 'MEN_ANEM0204', 'MEN_ANEM0509', 'MEN_ANEM10P', 'MEN_ANEM1019', 'MEN_ANEM2029', 'MEN_ANEM30P', 'PMEN', 'PMEN_ANEM0002', 'PMEN_ANEM0204', 'PMEN_ANEM0509', 'PMEN_ANEM10P', 'NBPI_RP_ANEM0002', 'NBPI_RP_ANEM0204', 'NBPI_RP_ANEM0509', 'NBPI_RP_ANEM10P', 'RP_PROP', 'RP_LOC', 'RP_LOCHLMV', 'RP_GRAT', 'NPER_RP', 'NPER_RP_PROP', 'NPER_RP_LOC', 'NPER_RP_LOCHLMV', 'NPER_RP_GRAT', 'ANEM_RP', 'ANEM_RP_PROP', 'ANEM_RP_LOC', 

(349759, 147)

In [38]:
histo_log_filtre = histo_log = histo_log[histo_log["DEP"].astype(str).isin(dep_filtre)]

In [39]:
histo_log_filtre.head()


,CODGEO,REG,DEP,LIBGEO,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,...,Plog_RP_LOCHLM,Plog_RP_LOCPRIV,Plog_RP_LOC,Plog_RP_PROP,Plog_RP_GRAT,Prp_RP_LOCHLM,Prp_RP_LOCPRIV,Prp_RP_LOC,Prp_RP_PROP,Prp_RP_GRAT
10873,30001,76,30,Aigaliers,351,230,90,31,329,19,...,0.0,65.5,8.5,52.7,4.3,0.0,100.0,13.0,80.4,6.5
10874,30002,76,30,Aigremont,415,328,78,9,391,21,...,0.0,79.0,12.8,64.6,1.4,0.0,100.0,16.2,81.7,1.8
10875,30003,76,30,Aigues-Mortes,5377,3984,1084,309,3810,1528,...,7.4,66.7,26.6,45.0,2.5,10.0,90.0,35.9,60.8,3.3
10876,30004,76,30,Aigues-Vives,1629,1418,90,121,1425,198,...,2.8,84.2,20.8,64.3,1.8,3.2,96.8,23.9,73.9,2.1
10877,30005,76,30,Aiguèze,251,110,114,27,189,54,...,0.0,43.8,10.0,31.5,2.4,0.0,100.0,22.7,71.8,5.5


In [40]:
histo_log_filtre.shape

(6931, 147)

In [41]:
# faire un table pour vois les modalités de la variable LIBGEO et le nombre de RP associé
table_libgeo = histo_log_filtre.groupby("LIBGEO")["RP"].sum().reset_index().sort_values(by="RP", ascending=False)
table_libgeo.head(20)

,LIBGEO,RP
362,Montpellier,1505595
388,Nîmes,712421
91,Béziers,363301
630,Sète,230434
15,Alès,215529
2,Agde,142734
308,Lunel,110517
204,Frontignan,102904
114,Castelnau-le-Lez,101317
45,Bagnols-sur-Cèze,85061


In [42]:
# Exporter le fichier final dans un fichier CSV
histo_log_filtre.to_csv("../SORTIE/Compil_clean.csv", index=False)

,insee_com,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791


,insee_com,LIBGEO,DEP,LOG,RP,RSECOCC,LOGVAC,MAISON,APPART,RP_1P,...,RP_ACH05,RP_ACH18,RP_ACH17,RP_ACH15,RP_ACH14,RP_ACH13,RP_ACH12,RP_ACH11,RP_ACH10,RP_LOCPRIV
0,30001,Aigaliers,30,350.745416,229.895704,89.912186,30.937526,329.380299,19.413995,4.875347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.213457
1,30002,Aigremont,30,414.761075,327.900205,77.663837,9.197033,390.869741,20.817543,4.165367,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.421945
2,30003,Aigues-Mortes,30,5376.776608,3983.513458,1084.436653,308.826498,3810.180887,1527.882255,73.796932,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1032.117277
3,30004,Aigues-Vives,30,1628.669871,1417.886360,90.191022,120.592489,1424.993904,197.521667,24.947336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,293.631063
4,30005,Aiguèze,30,250.965647,110.032157,113.718747,27.214743,188.938835,54.307282,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.116791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,34340,Villetelle,34,742.852859,681.218930,26.042505,35.591424,660.878825,76.405423,5.742286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146.739481
687,34341,Villeveyrac,34,1803.742847,1584.539764,79.514844,139.688239,1549.851329,250.958565,4.823179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378.706780
688,34342,Viols-en-Laval,34,106.725770,91.223301,9.301481,6.200988,79.568954,26.123318,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.922330
689,34343,Viols-le-Fort,34,608.641341,557.720784,9.792415,41.128142,504.717578,72.249401,5.947530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.668044
